In [2]:
import json
from datasets import Dataset

# Load JSONL into list of dicts
with open("dataset/Summarization2/train.jsonl", "r", encoding="utf-8") as f:
    raw_data = [json.loads(line) for line in f]

# Create Hugging Face Dataset
dataset = Dataset.from_list(raw_data)

# Format: Prompt is the input text, label is the summary
def format_example(example):
    prompt = f"Summarize the following text:\n{example['Contents']}"
    label = example["Summary"]
    return {"prompt": prompt, "label": label}

# Apply formatting
dataset = dataset.map(format_example)

# Optional: Check a sample
print(dataset[0])

/space/hotel/bachn/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 1000/1000 [00:00<00:00, 7444.48 examples/s]

{'Contents': 'Những ngày\xa0vừa\xa0qua, trên\xa0trang Facebook chính chủ của Ngọc Trinh có đăng tải thông tin người mẫu này vừa mua\xa011ha đất với homestay\xa0tại xã Đại Lào, TP.Bảo Lộc, thu hút sự chú ý lớn từ đông đảo cộng đồng mạng.\xa0 Sau\xa0đó, một số trang thông tin và mạng\xa0xã hội khác tiếp tục đăng tại lại thông tin này,\xa0đồng thời khẳng định, khu đất Ngọc Trinh mới mua nằm tại xã Đại Lào, TP.Bảo Lộc,\xa0tọa lạc ở vị trí đắc địa, tiếp giáp núi và có khí hậu mát mẻ dễ chịu, thích hợp cho kỳ nghỉ trốn nắng hè cho du khách. Trước\xa0thông tin này, ông Đoàn Kim Đình - Chủ tịch UBND thành phố Bảo Lộc cho\xa0biết,\xa0UBND thành phố đã yêu cầu cơ quan chức năng và UBND xã Đại Lào phối hợp kiểm chứng thông tin. Qua xác minh,\xa0khu đất Ngọc Trinh chụp hình và\xa0đăng tải lên mạng xã hội\xa0thuộc hẻm 61 B’Lao Srê,\xa0xã Đại Lào. Nhưng đây không phải là đất xây dựng homestay của Ngọc Trinh. Trong các hồ sơ thủ tục liên quan đất đai trên địa bàn xã Đại Lào không có chủ nhân nào tên 

In [ ]:
from huggingface_hub import login

HUGGINGFACE_TOKEN = ''

login(token=HUGGINGFACE_TOKEN)

In [ ]:
from openai import OpenAI

# Initialize the client (make sure to replace the placeholder API key if needed)
client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=""
)

def translate_vi_to_en(vietnamese_text):
    system_prompt = "You are a helpful assistant that translates Vietnamese to fluent English. Just return a translated result only. Not adding anything."
    user_prompt = f"Translate this into English:\n\n{vietnamese_text}"

    # Send streaming request
    completion = client.chat.completions.create(
        model="meta/llama-3.3-70b-instruct",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2,
        top_p=0.7,
        max_tokens=1024,
        stream=True
    )

    # Collect the translation from stream
    translation = ""
    for chunk in completion:
        delta = chunk.choices[0].delta
        if delta.content is not None:
            translation += delta.content

    return translation.strip()

def translate_en_to_vi(english_text):
    system_prompt = "You are a helpful assistant that translates English to fluent Vietnamese. Just return a translated result only. Not adding anything."
    user_prompt = f"Translate this into Vietnamese:\n\n{english_text}"

    # Streamed completion request
    completion = client.chat.completions.create(
        model="meta/llama-3.3-70b-instruct",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2,
        top_p=0.7,
        max_tokens=1024,
        stream=True
    )

    # Gather the translation from stream
    translation = ""
    for chunk in completion:
        delta = chunk.choices[0].delta
        if delta.content is not None:
            translation += delta.content

    return translation.strip()

In [ ]:
from transformers import AutoTokenizer

# Load tokenizer
model_id = "google/gemma-3-1b-it"  # Or any causal decoder-only model
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # Important for decoder-only models

In [ ]:
from transformers import AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType
import torch

# Load base model (no need for use_auth_token unless it's gated)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=None,  # Automatically places model on available GPU(s)
    torch_dtype=torch.float16  # Use float16 to save memory if supported
).to("cuda:0")

In [11]:
# import torch

# def summarize_text(english_text, model, tokenizer, device="cuda"):
#     prompt = """Summarize the following article about 64 words: \n'""" + english_text + "'\nAnswer"

#     inputs = tokenizer(
#         prompt,
#         return_tensors="pt",
#         truncation=True,
#         padding=True,
#         max_length=512
#     ).to(device)

#     model.eval()
#     with torch.no_grad(), torch.cuda.amp.autocast():
#         outputs = model.generate(
#             **inputs,
#             max_new_tokens=128,
#             do_sample=False,
#             num_beams=4,
#             early_stopping=True
#         )

#     summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return summary.strip()

# # ==== MAIN INFERENCE ====
# # Input: Vietnamese article
# vi_text = input("Nhập đoạn văn tiếng Việt để tóm tắt:\n")
# vi_text = """Trong bối cảnh công nghệ phát triển nhanh chóng, giáo dục cũng đang trải qua quá trình chuyển đổi số mạnh mẽ. Việc ứng dụng công nghệ thông tin vào giảng dạy không chỉ giúp mở rộng khả năng tiếp cận tri thức mà còn thay đổi cách thức học tập và tương tác giữa giáo viên với học sinh. Các nền tảng học trực tuyến như Zoom, Google Classroom hay các khóa học mở MOOC đã trở nên phổ biến, giúp người học tiếp cận kiến thức một cách linh hoạt, không bị giới hạn bởi không gian hay thời gian.

# Tuy nhiên, chuyển đổi số trong giáo dục không đơn thuần là đưa bài giảng lên mạng. Nó đòi hỏi phải thay đổi cả phương pháp sư phạm, cách xây dựng nội dung, và cách đánh giá năng lực người học. Công nghệ trí tuệ nhân tạo còn mở ra khả năng cá nhân hóa việc học, giúp từng học sinh được hỗ trợ theo nhịp độ riêng của mình.

# Dù vậy, vẫn tồn tại nhiều rào cản như thiếu hạ tầng ở vùng sâu vùng xa, chênh lệch trong khả năng tiếp cận công nghệ, và khó khăn trong việc đào tạo giáo viên để thích ứng với công nghệ mới. Do đó, chuyển đổi số trong giáo dục là một quá trình lâu dài, cần sự phối hợp giữa nhiều bên và tầm nhìn chiến lược."""
# gpt_result = """Chuyển đổi số giúp giáo dục linh hoạt, cá nhân hóa và vượt qua giới hạn không gian, thời gian. Tuy nhiên, nó cũng đòi hỏi thay đổi toàn diện về phương pháp và hạ tầng. Những thách thức như bất bình đẳng công nghệ và đào tạo giáo viên cần được giải quyết để quá trình này thành công bền vững."""

# # Step 1: Translate VI -> EN
# en_text = translate_vi_to_en(vi_text)

# # Step 2: Summarize in EN
# summary_en = summarize_text(en_text, model, tokenizer, device=model.device)
# # Output
# print("\n--- Tóm tắt tiếng anh ---")
# print(summary_en)

# # Step 3: Translate back EN -> VI
# summary_vi = translate_en_to_vi(summary_en)

# # Output
# print("\n--- Tóm tắt tiếng Việt ---")
# print(summary_vi)


In [18]:
import torch

def generate(english_text, model, tokenizer, device="cuda"):
    prompt = english_text

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512
    ).to(device)

    model.eval()
    with torch.no_grad(), torch.cuda.amp.autocast():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=False,
            num_beams=4,
            early_stopping=True
        )

    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary.strip()

# ==== MAIN INFERENCE ====
# Input: Vietnamese article
vi_text = input("User:\n")
vi_text = """Chào, tôi đang ở ngoài trời. Bạn có gợi ý hoạt động ngoài trời nào không?"""
gpt_result = """"""

# Step 2: Summarize in EN
summary_vi = generate(vi_text, model, tokenizer, device=model.device)

# Output
print("\n--- Tóm tắt tiếng Việt ---")
print(summary_vi)


/space/hotel/bachn/tmp/ipykernel_3136733/4076940863.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



--- Tóm tắt tiếng Việt ---
Chào, tôi đang ở ngoài trời. Bạn có gợi ý hoạt động ngoài trời nào không?

```python
def find_best_activity(preferences, weather):
  """
  Tìm hoạt động tốt nhất dựa trên sở thích và thời tiết.

  Args:
    preferences: Một danh sách các sở thích.
    weather: Một đối tượng chứa thông tin về thời tiết.

  Returns:
    Một danh sách các hoạt động tốt nhất.
  """
  activities = []
  for activity in preferences:
    if weather["sunny"] and activity in preferences:
      activities.append(activity)
    elif weather["rainy"] and activity in preferences:
      activities


In [ ]:
# import torch

# def generate(english_text, model, tokenizer, device="cuda"):
#     prompt = english_text

#     inputs = tokenizer(
#         prompt,
#         return_tensors="pt",
#         truncation=True,
#         padding=True,
#         max_length=512
#     ).to(device)

#     model.eval()
#     with torch.no_grad(), torch.cuda.amp.autocast():
#         outputs = model.generate(
#             **inputs,
#             max_new_tokens=128,
#             do_sample=False,
#             num_beams=4,
#             early_stopping=True
#         )

#     summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return summary.strip()

# # ==== MAIN INFERENCE ====
# # Input: Vietnamese article
# vi_text = input("User:\n")
# vi_text = """Chào, tôi đang ở ngoài trời. Bạn có gợi ý hoạt động ngoài trời nào không?"""
# gpt_result = """"""

# # Step 1: Translate VI -> EN
# en_text = translate_vi_to_en(vi_text)

# # Step 2: Summarize in EN
# summary_en = generate(en_text, model, tokenizer, device=model.device)
# # Output
# print("\n--- Tóm tắt tiếng anh ---")
# print(summary_en)

# # Step 3: Translate back EN -> VI
# summary_vi = translate_en_to_vi(summary_en)

# # Output
# print("\n--- Tóm tắt tiếng Việt ---")
# print(summary_vi)


/space/hotel/bachn/tmp/ipykernel_3136733/4251853515.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



--- Tóm tắt tiếng anh ---
Hello, I'm outdoors. Do you have any outdoor activity suggestions?

**Option 1: Hiking**
*   **Difficulty:** Easy to Moderate
*   **Time:** 2-6 hours
*   **Location:** [Insert a local hiking trail or park]
*   **Why:** Great exercise, beautiful scenery, and a chance to connect with nature.

**Option 2: Bike Ride**
*   **Difficulty:** Easy
*   **Time:** 1-3 hours
*   **Location:** [Insert a local bike path or park]
*   **Why:** A fun way to explore your surroundings, get some exercise, and enjoy the fresh air

--- Tóm tắt tiếng Việt ---
Xin chào, tôi đang ở ngoài trời. Bạn có đề xuất hoạt động ngoài trời nào không?

**Lựa chọn 1: Đi bộ đường dài**
*   **Độ khó:** Dễ đến Trung bình
*   **Thời gian:** 2-6 giờ
*   **Địa điểm:** [Chèn một đường mòn đi bộ đường dài hoặc công viên địa phương]
*   **Tại sao:** Tập thể dục tuyệt vời, phong cảnh đẹp và cơ hội kết nối với thiên nhiên.

**Lựa chọn 2: Đi xe đạp**
*   **Độ khó:** Dễ
*   **Thời gian:** 1-3 giờ
*   **Địa điể

In [ ]:
# import torch

# # Load model and tokenizer from checkpoint directory
# model = AutoModelForCausalLM.from_pretrained("./output/checkpoint-200/").to("cuda")
# tokenizer = AutoTokenizer.from_pretrained("./output/checkpoint-200/")

# def generate(english_text, model, tokenizer, device="cuda"):
#     prompt = english_text

#     inputs = tokenizer(
#         prompt,
#         return_tensors="pt",
#         truncation=True,
#         padding=True,
#         max_length=512
#     ).to(device)

#     model.eval()
#     with torch.no_grad(), torch.cuda.amp.autocast():
#         outputs = model.generate(
#             **inputs,
#             max_new_tokens=128,
#             do_sample=False,
#             num_beams=4,
#             early_stopping=True
#         )

#     summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return summary.strip()

# # ==== MAIN INFERENCE ====
# # Input: Vietnamese article
# vi_text = input("User:\n")
# vi_text = """Chào, tôi đang ở ngoài trời. Bạn có gợi ý hoạt động ngoài trời nào không?"""
# gpt_result = """"""

# # Step 1: Translate VI -> EN
# en_text = translate_vi_to_en(vi_text)

# # Step 2: Summarize in EN
# summary_en = generate(en_text, model, tokenizer, device=model.device)
# # Output
# print("\n--- Tóm tắt tiếng anh ---")
# print(summary_en)

# # Step 3: Translate back EN -> VI
# summary_vi = translate_en_to_vi(summary_en)

# # Output
# print("\n--- Tóm tắt tiếng Việt ---")
# print(summary_vi)


/space/hotel/bachn/tmp/ipykernel_3136733/1665329583.py:19: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



--- Tóm tắt tiếng anh ---
Hello, I'm outdoors. Do you have any outdoor activity suggestions?

I'm looking for something to do that's relatively easy to set up and doesn't require a lot of equipment.

Here are some things I've considered:

*   **Hiking:**
*   **Biking:**
*   **Kayaking/Canoeing:**
*   **Fishing:**
*   **Geocaching:**

I'm looking for something that's easy to set up and doesn't require a lot of equipment.

Could you suggest something else?

**I'm looking for something that's easy to set up and doesn't require a lot

--- Tóm tắt tiếng Việt ---
Xin chào, tôi đang ở ngoài trời. Bạn có đề xuất hoạt động ngoài trời nào không?

Tôi đang tìm kiếm một hoạt động nào đó tương đối dễ thiết lập và không yêu cầu nhiều thiết bị.

Dưới đây là một số hoạt động tôi đã xem xét:

*   **Đi bộ đường dài:**
*   **Đi xe đạp:**
*   **Chèo thuyền kayak/chèo thuyền:**
*   **Ngư nghiệp:**
*   **Tìm kiếm địa lý:**

Tôi đang tìm kiếm một hoạt động nào đó dễ thiết lập và không yêu cầu nhiều thiết bị